In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('eng_dataset.csv')

In [3]:
df.head()

,ID,sentiment,content
0,10941,anger,At the point today where if someone says somet...
1,10942,anger,@CorningFootball IT'S GAME DAY!!!! T MIN...
2,10943,anger,This game has pissed me off more than any othe...
3,10944,anger,@spamvicious I've just found out it's Candice ...
4,10945,anger,@moocowward @mrsajhargreaves @Melly77 @GaryBar...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7102 entries, 0 to 7101
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         7102 non-null   int64 
 1   sentiment  7102 non-null   object
 2   content    7102 non-null   object
dtypes: int64(1), object(2)
memory usage: 166.6+ KB


In [5]:
type(df)

pandas.core.frame.DataFrame

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [68]:
#selecting our features where maximum 90% of the things in the target and also minimum of 10 times mentioned
cv = CountVectorizer(max_df=0.9,min_df=10,stop_words='english')

In [69]:
dtm = cv.fit_transform(df['content'])

In [70]:
dtm

<7102x1001 sparse matrix of type '<class 'numpy.int64'>'
	with 32255 stored elements in Compressed Sparse Row format>

In [71]:
dtm[0]

<1x1001 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [72]:
from sklearn.decomposition import LatentDirichletAllocation

In [73]:
LDA = LatentDirichletAllocation(n_components=4,random_state=42)

In [74]:
LDA.fit(dtm)

LatentDirichletAllocation(n_components=4, random_state=42)

In [75]:
len(cv.get_feature_names_out())

1001

In [76]:
import random

In [77]:
for i in range(10):
    rand_wrd = random.randint(0,1000)
    print(cv.get_feature_names_out()[rand_wrd])

hurts
took
living
anger
ask
tired
smile
hand
yo
response


In [78]:
len(LDA.components_)

4

## For Topic 0

In [79]:
LDA.components_[0]

array([22.72692355,  0.25635503,  9.04494091, ...,  0.25053454,
        0.25290465, 14.86329989])

In [80]:
#length of each of the 4 components
len(LDA.components_[0])

1001

In [81]:
sin_topic = LDA.components_[0]

In [82]:
#Number of times argument is being mentioned.
sin_topic.argsort()

array([536, 579, 108, ..., 350, 635, 462])

In [83]:
#word with least representation
print(f'least rep: {sin_topic[579]}')

#word with most representation
print(f'Most rep: {sin_topic[462]}')

least rep: 0.25003340619374426
Most rep: 210.61389558346852


In [84]:
#Top 10 words
top10wrds = sin_topic.argsort()[-10:]

In [85]:
for index in top10wrds:
    print(cv.get_feature_names_out()[index])

want
time
life
new
really
happy
day
good
people
just


## For Each Topic

In [86]:
#go through anything thats immutable
for index, topic in enumerate(LDA.components_):
    print(f'Topic {index} top 15 words')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Topic 0 top 15 words
['angry', 'afraid', 'sober', 'nightmare', 'nervous', 'want', 'time', 'life', 'new', 'really', 'happy', 'day', 'good', 'people', 'just']


Topic 1 top 15 words
['week', 'people', 'way', 'life', 'love', 'depression', 'anxiety', 'start', 'going', 'just', 'smile', 'feel', 'make', 'don', 'like']


Topic 2 top 15 words
['bully', 'did', 'man', 'anger', 'let', 'didn', 'terrorism', 'need', 'fear', 'today', 'think', 'just', 'like', 'know', 'don']


Topic 3 top 15 words
['ly', 'broadcast', 'glee', 'home', 'lively', 'just', 'hilarious', 'time', 'live', 'amazing', 'lost', 've', 'watch', 'amp', 'ðÿ']




In [87]:
dtm.shape

(7102, 1001)

In [88]:
dtm

<7102x1001 sparse matrix of type '<class 'numpy.int64'>'
	with 32255 stored elements in Compressed Sparse Row format>

In [89]:
len(df)

7102

In [90]:
topic_results = LDA.transform(dtm)

In [91]:
topic_results.shape

(7102, 4)

In [103]:
topic_results[0].round(2)

array([0.04, 0.24, 0.69, 0.04])

In [104]:
topic_results[0].argmax()

2

In [105]:
df.head()

,ID,sentiment,content
0,10941,anger,At the point today where if someone says somet...
1,10942,anger,@CorningFootball IT'S GAME DAY!!!! T MIN...
2,10943,anger,This game has pissed me off more than any othe...
3,10944,anger,@spamvicious I've just found out it's Candice ...
4,10945,anger,@moocowward @mrsajhargreaves @Melly77 @GaryBar...


In [107]:
topic_results.argmax(axis=1)

array([2, 0, 0, ..., 1, 0, 1])

In [108]:
df['topic']= topic_results.argmax(axis=1)

In [109]:
df.head(10)

,ID,sentiment,content,topic
0,10941,anger,At the point today where if someone says somet...,2
1,10942,anger,@CorningFootball IT'S GAME DAY!!!! T MIN...,0
2,10943,anger,This game has pissed me off more than any othe...,0
3,10944,anger,@spamvicious I've just found out it's Candice ...,3
4,10945,anger,@moocowward @mrsajhargreaves @Melly77 @GaryBar...,3
5,10946,anger,@moocowward @mrsajhargreaves @Melly77 @GaryBar...,2
6,10947,anger,wanna go home and focus up on this game . Don'...,1
7,10948,anger,@virginmedia I've been disconnected whilst on ...,3
8,10949,anger,@virginmedia I've been disconnected whilst on ...,3
9,10950,anger,I wanna see you smile I don't wanna see you ma...,1


In [110]:
df['topic'].value_counts()

2    1861
0    1857
1    1857
3    1527
Name: topic, dtype: int64

In [111]:
df['sentiment'].value_counts()

fear       2252
anger      1701
joy        1616
sadness    1533
Name: sentiment, dtype: int64

In [115]:
fear_topic_list = [0,0,0,0]
for header,row in df.iterrows():
    if row['sentiment']=='fear':
        fear_topic_list[row['topic']] += 1
        
fear_topic_list

[551, 618, 682, 401]

In [117]:
sadness_topic_list = [0,0,0,0]
for header,row in df.iterrows():
    if row['sentiment']=='sadness':
        sadness_topic_list[row['topic']] += 1
        
sadness_topic_list

[432, 447, 380, 274]

In [118]:
joy_topic_list = [0,0,0,0]
for header,row in df.iterrows():
    if row['sentiment']=='joy':
        joy_topic_list[row['topic']] += 1
        
joy_topic_list

[366, 413, 260, 577]